The MaterialsProject provides magnetism info in two ways (https://docs.materialsproject.org/methodology/materials-methodology/magnetic-properties).
We only want to analyze those structures with magnetic structures determined by the magnetic orderings workflow, *not* those that are given as FM by default.
Jason Munro provided us with a list of MP task ids (data/find_query.json) related to the magnetic orderings workflow, which we map here to mp ids and filter for unique entries (unique by StructureMatcher).
In the next files within this folder, the statistical analysis as in ../MP is conducted on the magnetic structures to analyze bond angle occurrences as a function of the spin angle between neighboring magnetic sites.

In [1]:
from collections import Counter
import json
from mp_api.client import MPRester
import pandas as pd
from pymatgen.analysis.structure_matcher import StructureMatcher

/home/kueltzen/miniconda3/envs/kga_modnet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module named 'phonopy'
No module named 'phonopy'


In [2]:
# Get task ids
with open("data/find_query.json") as f:
    t = f.read()
t = str(t)
t = t.removeprefix('{\n    "task_id" : ')
t = t.removesuffix('\n}\n')
task_ids = [chunk.removeprefix('"').removesuffix('"') for chunk in t.split('\n}\n{\n    "task_id" : ')]
print(len(task_ids))
# Create sublists of max. query length
max_len = 5000
sublists = []
for i in range(0, len(task_ids), max_len):
    sublists.append(task_ids[i:i + max_len])

70582


In [21]:
# Loose check: magnetic orderings workflow last calc
calc_dirs = []
with MPRester("your-key") as mpr:
    for sl in sublists:
        docs = mpr.tasks.search(task_ids=sl, fields=['calcs_reversed'])
        calc_dirs.extend([r.calcs_reversed for r in docs])

Counter("magnetic" in ca[0].dir_name for ca in calc_dirs)

/tmp/ipykernel_601824/71064226.py:5: DeprecationWarning: Accessing tasks data through MPRester.tasks is deprecated. Please use MPRester.materials.tasks instead.
  docs = mpr.tasks.search(task_ids=sl, fields=['calcs_reversed'])
Retrieving TaskDoc documents: 100%|██████████| 582/582 [00:00<00:00, 7179661.55it/s]


Counter({True: 70582})

In [14]:
mp_ids = []
with MPRester("your-key") as mpr:
    for sl in sublists:
        docs = mpr.materials.search(task_ids=sl, fields=["material_id"])
        mp_ids.extend([str(d.material_id) for d in docs])
print(len(mp_ids))
mp_ids = set(mp_ids)
len(mp_ids)

Retrieving MaterialsDoc documents: 100%|██████████| 458/458 [00:00<00:00, 9018738.18it/s]

65465


5681

In [15]:
max_len = 5000
sublists_s = []
mp_ids = list(mp_ids)
for i in range(0, len(mp_ids), max_len):
    sublists_s.append(mp_ids[i:i + max_len])

mag_structs = {}
with MPRester("your-key") as mpr:
    for sl in sublists_s:
        structure_docs = mpr.materials.search(material_ids=sl, fields=["material_id", "structure"])
        mag_structs.update({str(sd.material_id): sd.structure for sd in structure_docs})
len(mag_structs)

Retrieving MaterialsDoc documents: 100%|██████████| 681/681 [00:00<00:00, 12527723.79it/s]


5681

In [22]:
mag_structs_dict = {k: v.as_dict() for k, v in mag_structs.items()}
with open("data/250512_5681_mag_structs.json", "w") as f:
    json.dump(mag_structs_dict, f)

In [29]:
df = pd.DataFrame([mag_structs], index=["structure"]).T
df

,structure
mp-1002570,"[[ 0.74791306 1.5657974 -3.07847656] K, [-3...."
mp-1003311,"[[ 1.8996727 -2.46446167 -2.22674842] Ca, [ 3..."
mp-1043213,"[[3.97296647 1.55013664 5.7679676 ] Mg, [2.016..."
mp-1043865,"[[0.16107109 5.28621348 1.90886084] La, [2.576..."
mp-1046445,"[[ 0.24429493 4.94250765 -1.6975124 ] Sr, [3...."
...,...
mp-780876,"[[ 5.51669341 -4.92728019 5.05502669] Li, [ 9..."
mp-849295,"[[2.15604931 4.32920271 2.84273994] Fe, [0.000..."
mp-850215,"[[1.54345746 0.83652827 0.59033805] Li, [4.630..."
mp-850808,"[[ 7.21858281 -0.82532276 4.91076505] Li, [4...."


In [32]:
# StructureMatcher check
df["group_index"] = None
max_group_index = 0
s = StructureMatcher()
for idx, (row_id, row) in enumerate(df.iterrows()):
    # Assign multiples group index
    unique = True
    for row_id_comp, row_comp in df[:idx].iterrows():
        if s.fit(row_comp["structure"], row["structure"]):
            print(row_id, row_id_comp)
            unique = False
            df.at[row_id, "group_index"] = row_comp["group_index"]
            break

    if unique:
        max_group_index += 1
        df.at[row_id, "group_index"] = max_group_index

mp-1175280 mp-1175183
mp-1173961 mp-1173945
mp-1205582 mp-1274754
mp-1273949 mp-1205631
mp-1274025 mp-1273743
mp-1226351 mp-1213081
mp-1175326 mp-1175347
mp-1206645 mp-1218686
mp-1272521 mp-1218247
mp-1274758 mp-1272170
mp-1173929 mp-1173949
mp-1105126 mp-1105343
mp-1174091 mp-1174361
mp-1272303 mp-1227910
mp-1078354 mp-1274754
mp-1175268 mp-1175125
mp-1218746 mp-1078887
mp-1175290 mp-1175170
mp-1205375 mp-1272373
mp-1105390 mp-1272979
mp-1174540 mp-1174761
mp-1175204 mp-1175271
mp-1084823 mp-1208834
mp-1105241 mp-1105404
mp-1174498 mp-1174737
mp-1175318 mp-1175359
mp-1177868 mp-1272966
mp-1190609 mp-1273390
mp-1178424 mp-1272234
mp-1047774 mp-1254079
mp-1095170 mp-1276755
mp-1173879 mp-1173880
mp-1218821 mp-1272721
mp-1218837 mp-1205631
mp-1103865 mp-1105162
mp-1175171 mp-1175277
mp-1175175 mp-1175278
mp-1094055 mp-1076424
mp-1272755 mp-1177943
mp-1105140 mp-1272384
mp-1174382 mp-1174385
mp-1218388 mp-1272748
mp-1273649 mp-1273743
mp-1275262 mp-1272613
mp-1284127 mp-1177894
mp-1285188

In [36]:
df_unique = df.drop_duplicates(subset="group_index", keep="first")
len(df_unique)

4559

In [37]:
df_unique.drop(columns=["group_index"], inplace=True)
df_unique["structure"] = df_unique["structure"].apply(lambda s: s.as_dict())
df_unique.to_json(f"data/250512_unique_{len(df_unique)}_mag_structs.json")

/tmp/ipykernel_366691/1240583051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(columns=["group_index"], inplace=True)
/tmp/ipykernel_366691/1240583051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["structure"] = df_unique["structure"].apply(lambda s: s.as_dict())
